<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/processed/ct_preprocessing_ubs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
===================================================
Author: Chiaki Tachikawa
Role: Data Science Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/chiaki-tachikawa
Date: 2025-03-03
Version: 1.1

Description:
    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:
    - Defining and applying a `preprocessor` function to clean and tokenize text data.
    - Reading and preprocessing various CSV files containing transcript data.
    - Segmenting text by bank name
    - Pairing question and answer by traditional data engineering techniques.
    - Cleaning Texts
    - Reorganising Data Frame
    - Exporting the preprocessed data to new CSV files for further analysis.

===================================================
"""

'\n===================================================\nAuthor: Chiaki Tachikawa\nRole: Data Science Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/chiaki-tachikawa\nDate: 2025-03-03\nVersion: 1.1\n\nDescription:\n    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:\n    - Defining and applying a `preprocessor` function to clean and tokenize text data.\n    - Reading and preprocessing various CSV files containing transcript data.\n    - Segmenting text by bank name\n    - Pairing question and answer by traditional data engineering techniques.\n    - Cleaning Texts\n    - Reorganising Data Frame\n    - Exporting the preprocessed data to new CSV files for further analysis.\n\n===================================================\n'

# **Library**

In [ ]:
#!pip install python-dotenv

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from google.colab import drive
import openai
import json
from dotenv import load_dotenv
import os
from google.colab import userdata
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# **Function**

**preprocessor function** : The function modifies the DataFrame data in place, adding two new columns (col1 and col2) with preprocessed text.


Input:
  - name of dataframe
  - name of column which contains the text to clean
  - name of column which is tokenized
  - name of column which is cleaned

In [ ]:
#create function to preprocess data
def preprocessor (data, col, col1,col2):
  #Copy col1umn
  data[col1]=data[col]
  data[col2]=data[col]


  #Adding column1
  #Lower the lettercase
  data[col1] = data[col1].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col1] = data[col1].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col1] = data[col1].apply(nltk.word_tokenize)

  #Remove numbers
  data[col1] = data[col1].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col1] = data[col1].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col1] = data[col1].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col1] = data[col1].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])

  #lemmatization
  lemmatizer = WordNetLemmatizer()
  data[col1] = data[col1].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])



  #Adding column2
  #Lower the lettercase
  data[col2] = data[col2].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col2] = data[col2].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #remove symbols
  data[col2] = data[col2].apply (lambda x: [re.sub(r"[.,'?]", "", x)])

  return


In [ ]:
# Function to extract names
def extract_name(full_string):
    return full_string.split(',')[0]

## **Structuring Data Frame**
Ensuring that each row has utterance from one analyst and exective and categorised by bank name to pair question and answer later.

In [ ]:
#drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#Defining qa_data
ubs_qa_data = pd.read_csv("/content/ubs_qna_section.csv")
#Check the imported data
ubs_qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file
0,Unknown,NaN,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
1,Sergio P. Ermotti,NaN,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
2,Chris Hallam,Goldman Sachs,"Very clear. Thanks. Kian Abouhossein, JPMorgan...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
3,Sergio P. Ermotti,NaN,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
4,Sarah Youngwood,NaN,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf


In [ ]:
"""
This code searches for Analyst / Exective name and their bank name in the 'utterance' column of a DataFrame and stores any matches in a new 'dummy' column. If no matches are found, the 'dummy' column remains None for that row.

"""
#Defining Analyst name and Bank Name with regex
pattern = r'\b[A-Z][a-z]+ [A-Z][a-z]+, [A-Z][A-Za-z]+'

#Looking for Analyst name and bank name from utterance to store it in dummy column
ubs_qa_data["dummy"]=None
for i in range(len(ubs_qa_data)):
  matches = re.findall(pattern, str(ubs_qa_data['utterance'][i]))
  if matches:
    ubs_qa_data.at[i, 'dummy'] = matches
  else:
    continue

#Defining lists of Exective names from UBS
exective_name = ["Sergio P. Ermotti","Sarah Youngwood	"]
exect_name = r'\b(Sergio P\. Ermotti|Sarah Youngwood)\b'

#Looking for Exective name from utterance to store it in ex_dummy column
ubs_qa_data["ex_dummy"]=None
for i in range(len(ubs_qa_data)):
  matches1 = re.findall(exect_name, str(ubs_qa_data["utterance"][i]))
  if matches1:
    ubs_qa_data.at[i,"ex_dummy"]=matches1
    ubs_qa_data.loc[i,"ex_dummy"].append("UBS")
  else:
    continue


In [ ]:
#Check if matched analyst name and exective name are populated in dummy and ex_dummy columns
ubs_qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy
0,Unknown,NaN,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Chis Hallam, Goldman]",None
1,Sergio P. Ermotti,NaN,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
2,Chris Hallam,Goldman Sachs,"Very clear. Thanks. Kian Abouhossein, JPMorgan...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
3,Sergio P. Ermotti,NaN,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
4,Sarah Youngwood,NaN,"So, when we give you the 74%, we focused inten...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None


In [ ]:
"""
This code calculates the number of rows with non-null values in the 'dummy' column and the total number of rows in the DataFrame.
"""
additional_row = len(ubs_qa_data) - ubs_qa_data"].isnull().sum()
print("The number of new rows: ", additional_row)
total = additional_row + len(ubs_qa_data("Total number of rows: ", total))

The number of new rows:  56
Total number of rows:  365


In [ ]:
"""
This code searches for Analyst/Exective name and their bank name in the 'utterance' column of a DataFrame, updates the 'speaker' column for the first row with matches, and splits the 'utterance' and inserts a new row for subsequent matches. If no matches are found, the loop continues to the next row.
Here are the steps in the code:
1, Defining matches with Bank name and Exective name in each utterance.
2, Create 3 conditions as follows.
  a, if it is the first row
    > Added Analyst and Bank name in speaker column and job_title column
  b, if there is bank name in the utterance
    I, when there is one match in the utterance
    > Created new row between the current and next index to split the utterance by each analyst
    II, when there are two matches in the utterance
    > Create new multiple rows between the current and next index to split the utterance by each analyst
  c, if there is Exective name in the utterance
    > Create a new row between the current and next index to split the utterance by each exective
"""


for  i in ubs_qa_data.index:
  matches = re.findall(pattern, str(ubs_qa_data['utterance'][i]))
  matches1 = re.findall(exect_name, str(ubs_qa_data['utterance'][i]))
  if matches and i==0:
    ubs_qa_data.at[i, 'speaker'] = matches
    ubs_qa_data.at[i,"job_title"]= matches
  elif matches:
    if len(ubs_qa_data["dummy"][i]) <2:
      new_index=i+0.5
      parts1 = [part.strip() for part in ubs_qa_data['utterance'][i].split(matches[0])]
      ubs_qa_data.at[i, 'utterance'] = parts1[0]
      ubs_qa_data.loc[new_index] = {"speaker":matches,"job_title":matches,"utterance":parts1[1], "call_date":ubs_qa_data["call_date"][i], "financial_quarter":ubs_qa_data["financial_quarter"][i],"source_file":ubs_qa_data["source_file"][i], "dummy":None}
    else:
      parts1 = [part.strip() for part in ubs_qa_data['utterance'][i].split(matches[0])]
      parts2 = [part.strip() for part in ubs_qa_data['utterance'][i].split(matches[1])]
      parts3 = [part.strip() for part in parts1[1].split(matches[1])]
      ubs_qa_data.at[i, 'utterance'] = parts1[0]
      ubs_qa_data.loc[i+0.2] = {"speaker":matches[0],"job_title":matches[0],"utterance":parts3[0], "call_date":ubs_qa_data["call_date"][i], "financial_quarter":ubs_qa_data["financial_quarter"][i],"source_file":ubs_qa_data["source_file"][i], "dummy":None}
      ubs_qa_data.loc[i+0.4] = {"speaker":matches[1],"job_title":matches[1],"utterance":parts2[1], "call_date":ubs_qa_data["call_date"][i], "financial_quarter":ubs_qa_data["financial_quarter"][i],"source_file":ubs_qa_data["source_file"][i], "dummy":None}
  elif matches1:
    new_index=i+0.5
    parts2 = [part.strip() for part in ubs_qa_data['utterance'][i].split(matches1[0])]
    ubs_qa_data.at[i, 'utterance'] = parts2[0]
    ubs_qa_data.loc[new_index] = {"speaker":ubs_qa_data["ex_dummy"][i],"job_title":"UBS","utterance":parts2[1], "call_date":ubs_qa_data["call_date"][i], "financial_quarter":ubs_qa_data["financial_quarter"][i],"source_file":ubs_qa_data["source_file"][i], "dummy":None}
  else:
    continue


In [ ]:
"""
Reset index due to new rows
"""
ubs_qa_data=ubs_qa_data.sort_index().reset_index(drop=True)

In [ ]:
"""
This code checks if the 'speaker' column contains a list, splits the first element of the list at the comma, and updates the 'speaker' and 'job_title' columns accordingly. If the 'speaker' is not a list, the loop continues to the next row.

"""
for i in range(len(ubs_qa_data)):
  if isinstance(ubs_qa_data['job_title'][i], list):
    parts = [part.strip() for part in ubs_qa_data['speaker'][i][0].split(',')]
    ubs_qa_data.at[i, 'speaker'] = parts[0]
    ubs_qa_data.at[i, 'job_title'] = parts[1]

  else:
    continue



In [ ]:
#Check if utterance was split correctly, and new rows were created when there is matched analyst name in utterance.
ubs_qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Chis Hallam, Goldman]",None
1,Sergio P. Ermotti,NaN,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None
2,Chris Hallam,Goldman Sachs,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",None
3,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,NaN
4,Sergio P. Ermotti,NaN,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,None


In [ ]:
"""
Row 56: The text was not populated in the utterance due to irregular simbol in the original text, therefore, this needs to be concanated with the text in the next row.
"""
text = str(ubs_qa_data["speaker"][57]) +str(ubs_qa_data["job_title"][57]) +str(ubs_qa_data["utterance"][57])
ubs_qa_data.at[56,"utterance"]=text
ubs_qa_data=ubs_qa_data.drop(index=57)
ubs_qa_data.reset_index(drop=True, inplace=True)

"""
Row 72: Some words were detected as "Analyst and Bank name" in the text, therefore, this needs to be concanated with the text in the next row.
"""
text1 = str(ubs_qa_data["speaker"][73]) +str(ubs_qa_data["job_title"][73]) +str(ubs_qa_data["utterance"][73])
ubs_qa_data.at[72,"utterance"]= ubs_qa_data["utterance"][72]+text1
ubs_qa_data=ubs_qa_data.drop(index=73)
ubs_qa_data.reset_index(drop=True, inplace=True)

"""
Row 2: Oringinal text shows Goldman Sachs, therefore, this needs to be concanated with the text in the next row.
"""
ubs_qa_data.at[2,"job_title"]="Goldman"

#Remove last row
ubs_qa_data=ubs_qa_data.drop(index=len(ubs_qa_data)-1)
ubs_qa_data.reset_index(drop=True, inplace=True)

#set speaker in row 30 and 31
ubs_qa_data.at[30,"job_title"]= "RBC"
ubs_qa_data.at[31,"job_title"]="Barclays"

In [ ]:
#Check if all bugs are fixed
ubs_qa_data[50:80]

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy
50,Alastair Ryan,Bank of America,"Yes, yes. Thank you, Sergio.",7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,None,None
51,Sergio P. Ermotti,NaN,"Well, enjoy -- enjoy your time off going forwa...",7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,None,None
52,Todd Tuckner,NaN,"Hey, Alastair. Thanks for the questions. So on...",7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,None,None
53,Alastair Ryan,Bank of America,"Okay. Thank you. And Sergio, thank you.",7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,None,None
54,Sergio P. Ermotti,NaN,Sure. Pleasure.,7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,None,None
55,Chris Hallam,Goldman Sachs,"Yeah. So two for me, by the end of the year, I...",7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,None,None
56,Todd Tuckner,NaN,"Yeah. Hey ChrisI’ll take the first so on, on w...",7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,None,None
57,Sergio P. Ermotti,NaN,"Yes, Chris, first of all, of course, you know,...",7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,None,None
58,Chris Hallam,Goldman Sachs,Right. Thank you.,7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,"[Kian Abouhossein, JP]",None
59,Kian Abouhossein,JP,Morgan Yes. Thank you for taking my question. ...,7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,None,NaN


In [ ]:
"""
This code snippet categorizes rows in a DataFrame ubs_qa_data based on the job_title column.
This effectively groups consecutive rows with the same bank name together to help segmenting question and answer by bank later.
"""
ubs_qa_data["category"]=None
current_bank=ubs_qa_data["job_title"][0]
for i in range(len(ubs_qa_data)):
  if current_bank == ubs_qa_data["job_title"][i]:
    ubs_qa_data.at[i,"category"]=current_bank
  elif current_bank!=ubs_qa_data["job_title"][i] and ubs_qa_data["job_title"][i] is np.NaN:
    ubs_qa_data.at[i,"category"]=current_bank
  else:
    current_bank=ubs_qa_data["job_title"][i]
    ubs_qa_data.at[i,"category"]=current_bank


In [ ]:
"""
add ubs in nan in job title
"""
for i in range(len(ubs_qa_data)):
  if ubs_qa_data["job_title"][i] is np.NaN:
    ubs_qa_data.at[i,"job_title"]="UBS"

In [ ]:
"""
Remove Caustionary statements at the end of transcript in each quarter.

"""
statement = r'\b(Cautionary Statement Regarding|Cautionary statement regarding)\b'
for i in range(len(ubs_qa_data)):
  matches = re.findall(statement, str(ubs_qa_data["utterance"][i]))
  if matches:
    parts = [part.strip() for part in ubs_qa_data['utterance'][i].split(matches[0])]
    ubs_qa_data.at[i, 'utterance'] = parts[0]
  else:
    continue

– closing remarks Okay. Looks like there are no more questions. Thank you for being with us this morning. As I said, despite the very uncertain market conditions, we delivered solid underlying results for the quarter. We have been maintaining our capital and liquidity positions, and we were also able to get very close to resolve a legacy matter that has been with us for 15 years. We discussed also the unique opportunity that the acquisition of Credit Suisse presents not only for our shareholders, but I believe also for all the rest of our stakeholders, employees, clients, society at large. And of course, this is not going to be a straight line journey. You know, it's going to be a challenging one, is a complex one. But I do believe that, you know, the benefits outweigh by far, all those factors. So, I know that you are looking for more details. We are also you are looking for more details. And we will really do our best to keep the communication channels open over the next few weeks an

In [ ]:
print(f'Here is the number of null value in the dataframe.: \n{ubs_qa_data.isnull().sum()}')

Here is the number of null value in the dataframe.: 
speaker                0
job_title              0
utterance              0
call_date              0
financial_quarter      0
source_file            0
dummy                309
ex_dummy             364
category               0
dtype: int64


In [ ]:
#Check if the data was segmented by bank
ubs_qa_data[130:150]

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category
130,Todd Tuckner,UBS,Yeah. That was our understanding from Credit S...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Mediobanca
131,Adam Terelak,Mediobanca,Okay. Thank you.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Mediobanca
132,Andrew Coombs,Citigroup,Good morning it’s Andrew Coombs from Citi and ...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Citigroup
133,Todd Tuckner,UBS,"So, Andrew, in terms of – I’ll take the second...",31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Citigroup
134,Andrew Coombs,UBS,Thank you.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Citigroup
135,Vishal Shah,Morgan Stanley,Hi. Thank you so much for your questions. My f...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Morgan Stanley
136,Todd Tuckner,UBS,"Hey, Vishal. I mean I think on this on that se...",31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Morgan Stanley
137,Vishal Shah,Morgan Stanley,Okay. Thank you so much.,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Morgan Stanley
138,Sergio P. Ermotti,UBS,Okay. The last answer and questions and I'm su...,31 August 2023,2Q23,2q23-earnings-call-remarks.pdf,None,None,Morgan Stanley
139,Giulia Miotto,Morgan Stanley,Hi. Good morning. Thank you for taking my ques...,14 August 2024,2Q24,2q24-earnings-call-remarks.pdf,None,None,Morgan Stanley


# **Pairing Question and Answer:**


In [ ]:
"""
Given that the ex_dummy column is no longer needed, next row's financial_quarter column is copied to the ex_dummy column.
"""
for i in range(len(ubs_qa_data)-1):
  ubs_qa_data.at[i,"ex_dummy"]=ubs_qa_data.at[i+1,"financial_quarter"]

In [ ]:
ubs_qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category
0,Chis Hallam,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Chis Hallam, Goldman]",1Q23,Goldman
1,Sergio P. Ermotti,UBS,"Okay. Thank you. On capital requirements, you ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Goldman
2,Chris Hallam,Goldman,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",1Q23,Goldman
3,Kian Abouhossein,JPMorgan,Yeah. Thanks. Just two questions. The first on...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,JPMorgan
4,Sergio P. Ermotti,UBS,"So, Sarah, take the first question. I'll take ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,JPMorgan


In [ ]:
"""
This code groups rows by bank name, and splits question and answer into separate columns.
Here are the steps;
1, Creating new columns to pair question and answer, and defining current bank name
2, Looping through each row in the DataFrame
3, Populating values in question and answer columns based on the following conditions:
  a, if the current row's financial quarter is same as the next row's.
  b, if the current row is the last row in the category.
  Otherwise, each row utterance concatenates the current answer or question
"""

ubs_qa_data["question"] = None
ubs_qa_data["answer"] = None
ubs_qa_data["analyst"] = None
ubs_qa_data["exective"] = None
current_bank=ubs_qa_data["category"][0]
analyst=[]
exective=[]
question=""
answer=""

for i in range(len(ubs_qa_data)):
  if ubs_qa_data["ex_dummy"][i]!=ubs_qa_data["financial_quarter"][i] and ubs_qa_data["job_title"][i]=="UBS":
      exective.append(str(ubs_qa_data["speaker"][i]))
      answer += str(ubs_qa_data["utterance"][i])
      ubs_qa_data.at[i,"question"]=question
      ubs_qa_data.at[i,"answer"]=answer
      ubs_qa_data.at[i,"analyst"]=analyst
      ubs_qa_data.at[i,"exective"]=exective
      exective=[]
      analyst=[]
      answer = ""
      question = ""
      ubs_qa_data.at[i,"category"]=current_bank
      if i<len(ubs_qa_data)-1:
        current_bank = ubs_qa_data["job_title"][i+1]
  else:
    if ubs_qa_data["job_title"][i]!="UBS" and current_bank==ubs_qa_data["category"][i]:
      question += str(ubs_qa_data["utterance"][i])
      analyst.append(str(ubs_qa_data["speaker"][i]))
    elif ubs_qa_data["job_title"][i]=="UBS" and current_bank==ubs_qa_data["category"][i]:
      answer += str(ubs_qa_data["utterance"][i])
      exective.append(str(ubs_qa_data["speaker"][i]))
    elif ubs_qa_data["job_title"][i]!="UBS" and current_bank!=ubs_qa_data["category"][i]:
      current_bank=ubs_qa_data["job_title"][i]
      analyst.append(str(ubs_qa_data["speaker"][i]))
      ubs_qa_data.at[i-1,"question"]=question
      ubs_qa_data.at[i-1,"answer"]=answer
      ubs_qa_data.at[i-1, "analyst"]=analyst
      ubs_qa_data.at[i-1,"exective"]=exective
      question = str(ubs_qa_data["utterance"][i])
      answer = ""
      analyst = []
      exective = []
    else:
      continue


In [ ]:
#Check if the last of each segmented bank contains question and answer
ubs_qa_data[10:60]

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category,question,answer,analyst,exective
10,Alastair Ryan,Bank of America,"Yeah. Thank you. Good morning. Welcome back, S...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Bank of America,None,None,None,None
11,Sergio P. Ermotti,UBS,"Thank you, Ryan. It is good to be back to inte...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Bank of America,None,None,None,None
12,Alastair Ryan,Bank of America,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Bank of America,"Yeah. Thank you. Good morning. Welcome back, S...","Thank you, Ryan. It is good to be back to inte...","[Alastair Ryan, Flora Bocahut]",[Sergio P. Ermotti]
13,Flora Bocahut,Jefferies,Yes. Good morning. The first question I wanted...,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Jefferies,None,None,None,None
14,Sarah Youngwood,UBS,"So, on the first question in terms of the tren...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Jefferies,None,None,None,None
15,Sergio P. Ermotti,UBS,"Yeah. On the client side. I mean, if you look ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Jefferies,None,None,None,None
16,Flora Bocahut,Jefferies,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Jefferies,Yes. Good morning. The first question I wanted...,"So, on the first question in terms of the tren...","[Flora Bocahut, Andrew Coombs]","[Sarah Youngwood, Sergio P. Ermotti]"
17,Andrew Coombs,Citi,"Good morning. Two questions. Firstly, just on ...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Citi,None,None,None,None
18,Sarah Youngwood,UBS,"So, on the first quarter or the first question...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Citi,None,None,None,None
19,Andrew Coombs,Citi,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Citi,"Good morning. Two questions. Firstly, just on ...","So, on the first quarter or the first question...","[Andrew Coombs, Adam Terelak]",[Sarah Youngwood]


In [ ]:
"""
Adding multiple Analyst name and Exective name if more than 2 people are participating in the conversation
"""
for i in range(len(ubs_qa_data)):
  if isinstance(ubs_qa_data["exective"][i], list):
    ubs_qa_data["exective"][i]=list(set(ubs_qa_data["exective"][i]))
  else:
    continue


<ipython-input-153-10e399f74f6a>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  qa_data["exective"][i]=list(set(qa_data["exective"][i]))
<ipython-input-153-10e399f74f6a>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pa

In [ ]:
"""
Dropping Nill in question columns to remove duplicated rows
"""
ubs_qa_data= ubs_qa_data.dropna(subset=["question"])
ubs_qa_data.reset_index(drop=True, inplace=True)

In [ ]:
ubs_qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category,question,answer,analyst,exective
0,Chris Hallam,Goldman,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",1Q23,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...","Okay. Thank you. On capital requirements, you ...","[Chis Hallam, Chris Hallam, Kian Abouhossein]",[Sergio P. Ermotti]
1,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,JPMorgan,Yeah. Thanks. Just two questions. The first on...,"So, Sarah, take the first question. I'll take ...","[Kian Abouhossein, Kian Abouhossein, Alastair ...","[Sergio P. Ermotti, Sarah Youngwood]"
2,Alastair Ryan,Bank of America,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Bank of America,"Yeah. Thank you. Good morning. Welcome back, S...","Thank you, Ryan. It is good to be back to inte...","[Alastair Ryan, Flora Bocahut]",[Sergio P. Ermotti]
3,Flora Bocahut,Jefferies,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Jefferies,Yes. Good morning. The first question I wanted...,"So, on the first question in terms of the tren...","[Flora Bocahut, Andrew Coombs]","[Sarah Youngwood, Sergio P. Ermotti]"
4,Andrew Coombs,Citi,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Citi,"Good morning. Two questions. Firstly, just on ...","So, on the first quarter or the first question...","[Andrew Coombs, Adam Terelak]",[Sarah Youngwood]


# **Cleaning Text and rearrangeing data frame:**
Cleaning each question and answer by removing stop words and symbols and lowering lettercase


In [ ]:
#preprocessing data
preprocessor(ubs_qa_data, "answer", "question_tokenised_data", "question_cleaned")
preprocessor(ubs_qa_data,"question","answer_tokenised_data","answer_cleaned")

In [ ]:
ubs_qa_data.head()

,speaker,job_title,utterance,call_date,financial_quarter,source_file,dummy,ex_dummy,category,question,answer,analyst,exective,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned
0,Chris Hallam,Goldman,Very clear. Thanks.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[Kian Abouhossein, JPMorgan]",1Q23,Goldman,"Chis Hallam, Goldman Sachs Yes. Good morning, ...","Okay. Thank you. On capital requirements, you ...","[Chis Hallam, Chris Hallam, Kian Abouhossein]",[Sergio P. Ermotti],"[okay, thank, you, capital, requirement, know,...",[okay thank you capital requirements know situ...,"[chi, hallam, goldman, sachs, yes, good, morni...",[chis hallam goldman sachs yes good morning ev...
1,Sergio P. Ermotti,UBS,"So on client retention, I – maybe let me reite...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,JPMorgan,Yeah. Thanks. Just two questions. The first on...,"So, Sarah, take the first question. I'll take ...","[Kian Abouhossein, Kian Abouhossein, Alastair ...","[Sergio P. Ermotti, Sarah Youngwood]","[sarah, take, first, question, take, secondso,...",[so sarah take first question take secondso gi...,"[yeah, thanks, two, question, first, one, rela...",[yeah thanks two questions first one related b...
2,Alastair Ryan,Bank of America,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Bank of America,"Yeah. Thank you. Good morning. Welcome back, S...","Thank you, Ryan. It is good to be back to inte...","[Alastair Ryan, Flora Bocahut]",[Sergio P. Ermotti],"[thank, you, ryan, good, back, interact, well,...",[thank you ryan good back interact well look y...,"[yeah, thank, you, good, morning, welcome, bac...",[yeah thank you good morning welcome back serg...
3,Flora Bocahut,Jefferies,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Jefferies,Yes. Good morning. The first question I wanted...,"So, on the first question in terms of the tren...","[Flora Bocahut, Andrew Coombs]","[Sarah Youngwood, Sergio P. Ermotti]","[first, question, term, trend, april, really, ...",[so first question terms trends april really s...,"[yes, good, morning, first, question, wanted, ...",[yes good morning first question wanted ask re...
4,Andrew Coombs,Citi,Thank you.,25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,None,1Q23,Citi,"Good morning. Two questions. Firstly, just on ...","So, on the first quarter or the first question...","[Andrew Coombs, Adam Terelak]",[Sarah Youngwood],"[first, quarter, first, question, seen, signif...",[so first quarter first question seen signific...,"[good, morning, two, question, firstly, slide,...",[good morning two questions firstly slide 10 r...


In [ ]:
#rename column
ubs_qa_data.rename(columns={"financial_quarter":"Quarter","question":"Question", "category":"Analyst_Bank", "answer": "Response", "exective":"Executive", "question_cleaned": "Question_cleaned", "answer_cleaned": "Response_cleaned", "source_file":"filename"},inplace=True)

In [ ]:
#reorganise column
ubs_qa_data=ubs_qa_data[["filename","Quarter","Question","Question_cleaned","Analyst_Bank","Response","Response_cleaned", "Executive"]]

In [ ]:
#Check the last dataframe
ubs_qa_data.head()

,filename,Quarter,Question,Question_cleaned,Analyst_Bank,Response,Response_cleaned,Executive
0,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",[okay thank you capital requirements know situ...,Goldman,"Okay. Thank you. On capital requirements, you ...",[chis hallam goldman sachs yes good morning ev...,[Sergio P. Ermotti]
1,1q23-earnings-call-remarks.pdf,1Q23,Yeah. Thanks. Just two questions. The first on...,[so sarah take first question take secondso gi...,JPMorgan,"So, Sarah, take the first question. I'll take ...",[yeah thanks two questions first one related b...,"[Sergio P. Ermotti, Sarah Youngwood]"
2,1q23-earnings-call-remarks.pdf,1Q23,"Yeah. Thank you. Good morning. Welcome back, S...",[thank you ryan good back interact well look y...,Bank of America,"Thank you, Ryan. It is good to be back to inte...",[yeah thank you good morning welcome back serg...,[Sergio P. Ermotti]
3,1q23-earnings-call-remarks.pdf,1Q23,Yes. Good morning. The first question I wanted...,[so first question terms trends april really s...,Jefferies,"So, on the first question in terms of the tren...",[yes good morning first question wanted ask re...,"[Sarah Youngwood, Sergio P. Ermotti]"
4,1q23-earnings-call-remarks.pdf,1Q23,"Good morning. Two questions. Firstly, just on ...",[so first quarter first question seen signific...,Citi,"So, on the first quarter or the first question...",[good morning two questions firstly slide 10 r...,[Sarah Youngwood]


UBS Management Discussion

In [ ]:
ubs_manag_data = pd.read_csv("/content/ubs_management_discussion.csv")
ubs_manag_data.head()

,speaker,utterance,call_date,financial_quarter,source_file
0,Unknown,"Youngwood, Group Chief Financial Officer Inclu...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
1,Sergio P. Ermotti,"Thank you, Sarah, good morning, everyone. I am...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
2,Latin America.,"In Asset Management, the combination will impr...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
3,Sarah Youngwood,"Thank you, Sergio. Good morning, everyone. rel...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf
4,Group Chief Financial Officer,Including analyst Q&A session,7 May 2024,1Q24,1q24-earnings-call-remarks.pdf


In [ ]:
preprocessor( ubs_manag_data, "utterance", "question_tokenised_data", "Utterance_cleaned" )

In [ ]:
#Rename columns
ubs_manag_data.rename(columns={"financial_quarter":"Quarter","speaker":"Speaker","utterance":"Utterance"}, inplace=True)
ubs_manag_data.head()

,Speaker,Utterance,call_date,Quarter,source_file,question_tokenised_data,Utterance_cleaned
0,Unknown,"Youngwood, Group Chief Financial Officer Inclu...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[youngwood, group, chief, financial, officer, ...",[youngwood group chief financial officer inclu...
1,Sergio P. Ermotti,"Thank you, Sarah, good morning, everyone. I am...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[thank, you, sarah, good, morning, everyone, h...",[thank you sarah good morning everyone happy b...
2,Latin America.,"In Asset Management, the combination will impr...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[asset, management, combination, improve, posi...",[asset management combination improve position...
3,Sarah Youngwood,"Thank you, Sergio. Good morning, everyone. rel...",25 April 2023,1Q23,1q23-earnings-call-remarks.pdf,"[thank, you, sergio, good, morning, everyone, ...",[thank you sergio good morning everyone relate...
4,Group Chief Financial Officer,Including analyst Q&A session,7 May 2024,1Q24,1q24-earnings-call-remarks.pdf,"[including, analyst, session]",[including analyst q&a session]


In [ ]:
#Reorder the columns
ubs_manag_data=ubs_manag_data[["Quarter","Speaker","Utterance","Utterance_cleaned"]]
ubs_manag_data.head()

,Quarter,Speaker,Utterance,Utterance_cleaned
0,1Q23,Unknown,"Youngwood, Group Chief Financial Officer Inclu...",[youngwood group chief financial officer inclu...
1,1Q23,Sergio P. Ermotti,"Thank you, Sarah, good morning, everyone. I am...",[thank you sarah good morning everyone happy b...
2,1Q23,Latin America.,"In Asset Management, the combination will impr...",[asset management combination improve position...
3,1Q23,Sarah Youngwood,"Thank you, Sergio. Good morning, everyone. rel...",[thank you sergio good morning everyone relate...
4,1Q24,Group Chief Financial Officer,Including analyst Q&A session,[including analyst q&a session]


# **Export the output as a csv file**

UBS QA section

In [ ]:
#export preprocessed data
#preprocessed_qa_csv_path3 = "/content/ubs_qa_df_preprocessed.csv"
#processed_df.to_csv(preprocessed_qa_csv_path3, index=False)

UBS management discussion

In [ ]:
#export preprocessed data
#preprocessed_qa_csv_path4 = "/content/sample_data/ubs_management_df_preprocessed.csv"
#manag_data.to_csv("ubs_management_discussion.csv", index=False)